In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.sankey import Sankey

In [2]:
colunas = [
            "ValorTotal", "DocNum", "ObjType", "LctoContabil", "DocEntry", 
            "CardCode", "CardName", "GroupCode", "GroupName", "SlpCode", 
            "SlpName", "Data", "BPLId", "FilialName", "PesoTotal", 
            "U_TU_QuemEntrega", "Latitude", "Longitude", "Rua", "NumRua", 
            "Complemento", "Bairro", "Cidade", "Estado", "Cep"
]

force_text = [
            "ObjType", "LctoContabil", "CardCode", "CardName", 
            "GroupCode", "GroupName", "SlpCode", "SlpName", 
            "BPLId", "FilialName", "U_TU_QuemEntrega", "Latitude", 
            "Longitude", "Rua", "NumRua", "Complemento", "Bairro", 
            "Cidade", "Estado", "Cep"
]

force_numeric = ["ValorTotal", "PesoTotal"]
force_integer = ["DocNum", "DocEntry"]
force_date = ["Data"]


In [3]:
parquet_path = "data//analise_venda_head.parquet"

df = pd.read_parquet(parquet_path)

df.head(10)

,ValorTotal,DocNum,ObjType,LctoContabil,DocEntry,CardCode,CardName,GroupCode,GroupName,SlpCode,...,Latitude,Longitude,Rua,NumRua,Complemento,Bairro,Cidade,Estado,Cep,UpdateTime
0,5776.82,80787,13,406364,108863,C019677,MASTER SERRALHERIA LTDA,105.0,INDUSTRIALIZA GO,41.0,...,None,None,PRESIDENTE TANCREDO NEVES,445,QUADRA 07 LOTE 03A,SETOR PLANALTO,JATAI,GO,75805115,2025-07-21 08:19:12
1,321.71,18920,13,89826,27357,C000219,MAURILIO MACHADO VELOSO,102.0,CONS FINAL NAO CONT,2.0,...,None,None,TAMOYO,0,ESQUINA COM RUA ARAGUAIA,SANTO AGOSTINHO,RIO VERDE,GO,75904680,2023-07-14 15:54:48
2,2049.56,6724,13,31227,10896,C0020178,INSTITUTO DE PLANEJAMENTO DE SERVICOS ESPEC IPGSE,102.0,CONS FINAL NAO CONT,17.0,...,None,None,MANUEL DOS SANTOS,168,QUADRA N LOTE 12,PEDROLINA,SANTA HELENA DE GOIAS,GO,75920000,2023-04-04 17:57:49
3,492.66,34234,13,162138,47710,C013851,F METAL MONTAGEM E MANUTENCAO LTDA,105.0,INDUSTRIALIZA GO,2.0,...,None,None,PAU BRASIL,0,QUADRA53 LOTE 913 B,RESIDENCIAL VENEZA,RIO VERDE,GO,75910048,2023-12-08 10:43:11
4,65.00,82873,13,419591,111686,C0028853,SANTINHO DANIELEWZ,103.0,PRODUTOR RURAL,3.0,...,None,None,GO 174,0,SENTIDO AP DO RIO DOCE,ZONA RURAL,RIO VERDE,GO,75901970,2025-08-18 08:12:18
5,1106.22,72637,13,358513,97839,C008486,PEDRO HENRIQUE FORNOS ADRIAO LOURENCO 03759809111,104.0,CONSTRUTORA GO,28.0,...,None,None,SANTO ANTONIO,968,QUADRA 84 LOTE 16,VILA JARDIM RIO CLARO,JATAI,GO,75802021,2025-04-03 16:55:19
6,2125.50,48571,13,235373,66476,C000092,HIDRO VOLT MATERIAIS ELETRICOS E HIDRAULICOS LTDA,107.0,REVENDA,4.0,...,None,None,DAS TURMALINAS,339,QD 37 LT 04,PARQUE BANDEIRANTES,RIO VERDE,GO,None,2024-06-13 14:58:14
7,441.13,123664,15,136277,3197,C000955,EDUARDO RIBEIRO RALSTON,103.0,PRODUTOR RURAL,4.0,...,None,None,RIO VERDE,0,QD 08 LT 108,MORADA DA COLINA,MONTIVIDIU,GO,None,2023-10-11 14:33:15
8,1053.80,8748,13,41257,13631,C016181,ELISMAR ROSA DA SILVA 83238573187,104.0,CONSTRUTORA GO,17.0,...,None,None,223,0,QUADRA SD LOTE 18,JARDIM SANTA HELENA,SANTA HELENA DE GOIAS,GO,75920000,2023-03-27 08:41:24
9,338.00,43398,13,211731,59973,C015778,EDILSON PROFIRO DAMASCENO,102.0,CONS FINAL NAO CONT,17.0,...,None,None,17,0,QUADRA 9 LOTE 247,RESIDENCIAL ATALIA,RIO VERDE,GO,75912075,2024-04-11 10:42:39


In [7]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

parquet_path = "data//analise_venda_head.parquet"
df = pd.read_parquet(parquet_path)
df['Data'] = pd.to_datetime(df['Data'])


df_2025 = df[df['Data'].dt.year == 2025].copy()

# Tratamento para caso não haja dados em 2025
if df_2025.empty:
    print("Não foram encontrados dados para o ano de 2025.")
else:
    # --- 3. Preparação dos Dados para o Gráfico Sankey ---

    # Calcular o faturamento total de 2025
    total_faturamento_2025 = df_2025['ValorTotal'].sum()

    # Agregar dados por Grupo de Cliente (Source)
    source_data = df_2025.groupby('GroupName')['ValorTotal'].sum().reset_index()
    source_data.rename(columns={'ValorTotal': 'Valor'}, inplace=True)

    # Agregar dados por Vendedor (Target)
    target_data = df_2025.groupby('SlpName')['ValorTotal'].sum().reset_index()
    target_data.rename(columns={'ValorTotal': 'Valor'}, inplace=True)

    # Criar a lista de todos os nós (labels)
    all_labels = list(source_data['GroupName']) + ['Faturamento Total 2025'] + list(target_data['SlpName'])
    
    # Criar um dicionário para mapear label para índice
    label_to_index = {label: i for i, label in enumerate(all_labels)}
    
    # Criar os links
    # Fluxo 1: Grupo de Cliente -> Faturamento Total
    sources = [label_to_index[name] for name in source_data['GroupName']]
    targets = [label_to_index['Faturamento Total 2025']] * len(source_data)
    values = list(source_data['Valor'])
    
    # Fluxo 2: Faturamento Total -> Vendedor
    sources.extend([label_to_index['Faturamento Total 2025']] * len(target_data))
    targets.extend([label_to_index[name] for name in target_data['SlpName']])
    values.extend(list(target_data['Valor']))

    # --- 4. Formatação dos Rótulos dos Nós (com Valor e Porcentagem) ---
    
    # Dicionário com o valor total de cada nó
    node_values = {**pd.Series(source_data.Valor.values, index=source_data.GroupName).to_dict(),
                   **{'Faturamento Total 2025': total_faturamento_2025},
                   **pd.Series(target_data.Valor.values, index=target_data.SlpName).to_dict()}
                   
    # Função para formatar o valor monetário
    def format_currency(value):
        if value >= 1_000_000:
            return f'R$ {value/1_000_000:.1f}M'
        if value >= 1_000:
            return f'R$ {value/1_000:.1f}K'
        return f'R$ {value:.2f}'

    # Criar os rótulos formatados
    formatted_labels = []
    for label in all_labels:
        value = node_values[label]
        percentage = (value / total_faturamento_2025) * 100
        formatted_labels.append(f"{label}<br><b>{format_currency(value)} - {percentage:.1f}%</b>")

    # --- 5. Criação do Gráfico Sankey ---
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=25,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=formatted_labels,
            color="royalblue"
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values
        )
    )])

    fig.update_layout(
        title_text="Fluxo de Faturamento: Grupo de Clientes para Vendedores (Ano: 2025)",
        font_size=12,
        height=600 # Ajuste a altura conforme necessário
    )

    fig.show()